## Import library 

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

import matplotlib 
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestRegressor

# EconML
from econml.dml import LinearDMLCateEstimator, ForestDMLCateEstimator
from econml.cate_interpreter import SingleTreeCateInterpreter, SingleTreePolicyInterpreter


import sys
sys.path.insert(
    1,
    '/home/dtd/Documents/interpretable_machine_learning/Source Code/my_work/lib'
)
sys.path.insert(
    1,
    '/home/dtd/Documents/interpretable_machine_learning/Causal Inference/CEA')




from sklearn.preprocessing import LabelEncoder

import data_load
import numpy as np
import pandas as pd
import logging
import dowhy.datasets
import dowhy
import propensity_score_estimator as pse
import incremental_ps_score_estimator as ipse
import math
import timeit
import evaluation as evl
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import PolynomialFeatures

from econml.drlearner import ForestDRLearner
from econml.drlearner import LinearDRLearner
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
from econml.metalearners import SLearner
from econml.metalearners import XLearner
from econml.metalearners import TLearner

from econml.drlearner import LinearDRLearner
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from econml.dml import ForestDMLCateEstimator
from dowhy import CausalModel
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import sem
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from math import sqrt
from econml.dml import LinearDMLCateEstimator
from econml.inference import BootstrapInference
from econml.dml import SparseLinearDMLCateEstimator
from sklearn.linear_model import LassoCV, ElasticNetCV
from mlens.ensemble import SuperLearner
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import math
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

[MLENS] backend: threading


## Load data

In [2]:
df = pd.read_csv("/home/dtd/Documents/interpretable_machine_learning/Source Code/data/data.csv")

In [3]:
df.head()

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,offer,conversion
0,10,142.44,1,0,Surburban,0,Phone,Buy One Get One,0
1,6,329.08,1,1,Rural,1,Web,No Offer,0
2,7,180.65,0,1,Surburban,1,Web,Buy One Get One,0
3,9,675.83,1,0,Rural,1,Web,Discount,0
4,2,45.34,1,0,Urban,0,Web,Buy One Get One,0


In [4]:
df['treatment'] = np.where(df['offer'] == 'No Offer', 0, 1)
lb_make = LabelEncoder()
df['zip_code'] = lb_make.fit_transform(df['zip_code'])
df['channel'] =  lb_make.fit_transform(df['channel'])

In [5]:
treatment = "treatment"
outcome = "spend"
cov = ['recency', 'history', 'used_discount', 'used_bogo', 
       'zip_code','is_referral', 'channel', 'conversion']
col = ['recency', 'history', 'used_discount', 'used_bogo', 
       'zip_code','is_referral', 'channel', 
       'offer', 'conversion', 'treatment']

In [6]:
df.head()

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,offer,conversion,treatment
0,10,142.44,1,0,1,0,1,Buy One Get One,0,1
1,6,329.08,1,1,0,1,2,No Offer,0,0
2,7,180.65,0,1,1,1,2,Buy One Get One,0,1
3,9,675.83,1,0,0,1,2,Discount,0,1
4,2,45.34,1,0,2,0,2,Buy One Get One,0,1


In [7]:
df.treatment.value_counts()

1    42694
0    21306
Name: treatment, dtype: int64

In [8]:
#function for calculating the uplift
def calc_uplift(df):
    avg_order_value = 25
    
    #calculate conversions for each offer type
    base_conv = df[df.treatment == 0]['conversion'].mean()
    disc_conv = df[df.treatment == 1]['conversion'].mean()
    
    #calculate conversion uplift for discount and bogo
    disc_conv_uplift = disc_conv - base_conv
    
    #calculate order uplift
    disc_order_uplift = disc_conv_uplift * len(df[df.treatment == 1]['conversion'])
    
    #calculate revenue uplift
    disc_rev_uplift = disc_order_uplift * avg_order_value
    
    return np.round(disc_conv_uplift*100,2), np.round(disc_order_uplift,2), np.round(disc_rev_uplift,2)
    

### Benchmark 

In [9]:
conv, order, rev = calc_uplift(df)

In [10]:
print('Discount Conversion Uplift: {0}%'.format(np.round(conv*100,2)))
print('Discount Order Uplift: {0}'.format(np.round(order,2)))
print('Discount Revenue Uplift: ${0}\n'.format(np.round(rev,2)))
print('Revenue Uplift Per Targeted Customer: ${0}\n'.format(np.round(rev/df.shape[0],2)))

Discount Conversion Uplift: 609.0%
Discount Order Uplift: 2599.29
Discount Revenue Uplift: $64982.36

Revenue Uplift Per Targeted Customer: $1.02



## Model 

In [11]:
model_t = LogisticRegression()
model_t.fit(df[cov], df[treatment])

LogisticRegression()

In [12]:
df['p1'] = 1 - model_t.predict_proba(df[cov])[:,:1]
df['p0'] = model_t.predict_proba(df[cov])[:,:1]

In [13]:
df.head()

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,offer,conversion,treatment,p1,p0
0,10,142.44,1,0,1,0,1,Buy One Get One,0,1,0.654940,0.345060
1,6,329.08,1,1,0,1,2,No Offer,0,0,0.651035,0.348965
2,7,180.65,0,1,1,1,2,Buy One Get One,0,1,0.652539,0.347461
3,9,675.83,1,0,0,1,2,Discount,0,1,0.657202,0.342798
4,2,45.34,1,0,2,0,2,Buy One Get One,0,1,0.648013,0.351987


In [14]:
df.shape

In [15]:
from collections import Counter
delta_seq = np.random.uniform(low=0.01, high=1, size=(10000,))
rev_seq = []
dictionary_ = []
delta_result = []
cov_seq = []

ihdp: q1 => optimize outcome
data: q1 => binary => revenue()

for delta in tqdm(delta_seq):
    df['q1'] = (delta * df['p1']) / (delta * df['p1'] + df['p0'])
    df['treatment'] = np.where(df['q1'] >= 0.5, 1, 0)
    d = df.treatment.value_counts().to_dict()
    if 1 not in d:
        count = 0
    else:
        count = d[1]
    
    conv, order, revenue = calc_uplift(df)
    
    if not math.isnan(revenue):
        delta_result.append(delta)
        rev_seq.append(revenue)
        cov_seq.append(conv)
        if count not in dictionary_:
            dictionary_.append(count)


100%|██████████| 10000/10000 [01:34<00:00, 105.72it/s]


In [16]:
max_index = np.argmax(rev_seq)
delta = delta_result[max_index]

df['q1'] = (delta * df['p1']) / (delta * df['p1'] + df['p0'])
df['treatment'] = np.where(df['q1'] >= 0.5, 1, 0)
conv, order, revenue = calc_uplift(df)
conv, order, revenue

In [17]:
print('Discount Revenue Uplift: ${0}'.format(np.round(revenue,2)))
print('Discount Conversion Uplift: {0}%'.format(np.round(conv*100,2)))
print('Revenue Uplift Per Targeted Customer: ${0}'.format(np.round(revenue/df.shape[0],2)))

Discount Revenue Uplift: $234850.0
Discount Conversion Uplift: 10000.0%
Revenue Uplift Per Targeted Customer: $3.67


In [19]:
rev = np.round(234850.0/ 64982*100,2)
print("The revenue has been improved {}%".format(rev))

The revenue has been improved 361.41%


## Reference 
- https://towardsdatascience.com/market-response-models-baf9f9913298